In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn import svm
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

from LSSVMRegression import LSSVMRegression

from windTransformer import WindVectorTransformer
from windTransformer import WindDegreeTransformer
from windData import WindDataCollector

import matplotlib.pyplot as plt

import numpy as np
import math
import pandas as pd

from influxdb import InfluxDBClient # install via "pip install influxdb"

import datetime

In [7]:
def eval_metrics(actual, pred):
	rmse = np.sqrt(mean_squared_error(actual, pred))
	mae = mean_absolute_error(actual, pred)
	r2 = r2_score(actual, pred)
	return rmse, mae, r2


## Data prep

In [8]:
start_time = datetime.datetime(2021, 1, 1, 0, 0, 0).strftime("'%Y-%m-%dT%H:%M:%SZ'")


dataCollector = WindDataCollector()

gen_df = dataCollector.getGenerationData(now = start_time, delta="90")
wind_df = dataCollector.getWindData(now = start_time, delta="90")

gen_df_alligned = pd.merge_asof(wind_df,gen_df,left_index=True, right_index=True)

In [9]:
train_length = int(len(gen_df_alligned)*0.9)

train_X = gen_df_alligned.iloc[:train_length][[
#    "Direction",
    "Speed"]]
test_X = gen_df_alligned.iloc[train_length:][[
#    "Direction",
    "Speed"]]

train_y = gen_df_alligned.iloc[:train_length]["Total"]
test_y = gen_df_alligned.iloc[train_length:]["Total"]

In [10]:
import plotly.express as px


# fig = px.scatter(gen_df_alligned, x="Speed", y="Total")
# fig.show()

## Linear regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [27]:
pipeline = Pipeline(steps=[
    # ("column_transformers", ColumnTransformer([
#        ("direction_degree_transformer", WindDegreeTransformer(), ["Direction"]),
        # ("polynomial_features", PolynomialFeatures() , ["Speed"])
    ])),
	("mlp_model", MLPRegressor((32, 16), 
    activation="relu", 
    solver="sgd", 
    # batch_size=32,
    learning_rate_init=0.1, 
    max_iter=80,
    ))
])
parameters = {
	# 'column_transformers__polynomial_features__degree':[1,2,3,4,5,6,7,8]
}

SyntaxError: unmatched ')' (2029095877.py, line 5)

In [28]:
tscv = TimeSeriesSplit(n_splits=5)
pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=15, cv= tscv)

pipeline.fit(train_X, np.ravel(train_y))

bestParams = pipeline.best_params_

predicted_qualities = pipeline.predict(test_X)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print(bestParams)
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/model_selection/_search.py", line 875, in fit
    self._run_search(evaluate_candidates)
  File "/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/model_selection/_search.py", line 1379, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/model_selec

## Ridge Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [13]:
# pipeline = Pipeline(steps=[
#     ("column_transformers", ColumnTransformer([
# #        ("direction_degree_transformer", WindDegreeTransformer(), ["Direction"]),
#         ("polynomial_features", PolynomialFeatures() , ["Speed"])
#     ])),
# 	("linear_model", Ridge())
# ])
# parameters = {
#     'column_transformers__polynomial_features__degree':[1,2,3,4,5,6,7,8],
# 	"linear_model__alpha":[0.001, 0.01, 0.1, 1, 10, 100]
# }

In [14]:
# tscv = TimeSeriesSplit(n_splits=5)
# pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=15, cv= tscv)

# pipeline.fit(train_X, np.ravel(train_y))

# bestParams = pipeline.best_params_

# predicted_qualities = pipeline.best_estimator_.predict(test_X)

# (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

# print(bestParams)
# print("  RMSE: %s" % rmse)
# print("  MAE: %s" % mae)
# print("  R2: %s" % r2)

/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=6.28435e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=5.75415e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=2.95377e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=5.75415e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", over

{'column_transformers__polynomial_features__degree': 3, 'linear_model__alpha': 0.001}
  RMSE: 4.1577791423580965
  MAE: 3.2475000182302387
  R2: 0.896534397808669


/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.41105e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=3.36143e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.386e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/Thomas/opt/anaconda3/envs/wild/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=2.23241e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwr